In [ ]:
# External modules
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import importlib
import numpy as np
import os, sys
import json
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../utils/'))

# Local modules
import get_oms_data
from get_oms_data import get_oms_api, get_oms_data, get_oms_response_attribute

import json_utils as jsonu
import plot_utils as pu

import mplhep as hep
hep.style.use("CMS")

import json_utils
from refruns_utils import get_reference_run as RRfetch
import refrank_utils as rrr
import oms_utils as omsu

In [ ]:
# importlib.reload(get_oms_data)
importlib.reload(rrr)
# importlib.reload(omsu)

# Getting the Data
We start by instantiating the OMS API. The OMS API will be used to get Run and LS level data and will be the main source of information for the reference run ranking (RRR) system.

In [ ]:
omsapi = get_oms_api()

In [ ]:
# These are the features that will be fetched from OMS
# Run level attributes
run_attribs = [
    "run_number", #
    "init_lumi", #
    "recorded_lumi",
    "delivered_lumi",
    "energy",
    "end_lumi", #
    "hlt_physics_rate",
    "fill_number", # Neccesary for filtering NaNs
    "initial_prescale_index",
    "last_lumisection_number",
    "l1_rate",
    "hlt_physics_counter",
    "l1_hlt_mode",
    "duration"
]

# Lumisecion (LS) level attributes
ls_attribs = [
    "fill_number", # Neccesary for filtering NaNs
    "run_number",
    "lumisection_number",
    "pileup",
    "delivered_lumi",
    "recorded_lumi",
    "init_lumi",
    "end_lumi",
]

In [ ]:
# Using the API, you can check all of the available features
run_query = rrr.makeDF(get_oms_data(omsapi, "runs", 316201, limit_entries=1))
print("All available run level features:\n")
run_query.info()

In [ ]:
# Using the API, you can check all of the available features
ls_query = rrr.makeDF(get_oms_data(omsapi, "lumisections", 316201, limit_entries=1))
print("All available LS level features:\n")
ls_query.info()

## Loading data

In [ ]:
# Run we wish to certify and for which we will find a correspond reference run
# newest_run = 326201
# oldest_run = 309000

# All 2018 runs
# newest_run = 327802
# oldest_run = 308241

# Almost all 2018 runs
newest_run = 326201
oldest_run = 308800 # Oldest run OMS is allowing me to get

# Range of runs of interest
run_range = (oldest_run, newest_run)
print("Run range: {}".format(run_range))
print("Potential total number of runs to load: {}".format(newest_run - oldest_run))

In [ ]:
# Filters to apply to data fetched from OMS
run_filters = [
    {"attribute_name": "l1_hlt_mode", "value": "collisions", "operator": "LIKE"} # Only collision runs
]

In [ ]:
run_df, ls_df = omsu.get_runs_lss(run_range, omsapi, run_attribs, ls_attribs, run_filters=run_filters)

In [ ]:
# Optional: Save to parquet
# run_df.to_parquet('/eos/user/r/rcruzcan/SWAN_projects/RefRunRank/data/omsrundata_2018.parquet')
# ls_df.to_parquet('/eos/user/r/rcruzcan/SWAN_projects/RefRunRank/data/omslsdata_2018.parquet')

In [ ]:
# Optional: Load from parquet
run_df = pd.read_parquet('/eos/user/r/rcruzcan/SWAN_projects/RefRunRank/data/omsrundata_2018.parquet')
ls_df = pd.read_parquet('/eos/user/r/rcruzcan/SWAN_projects/RefRunRank/data/omslsdata_2018.parquet')

In [ ]:
# Quick check of the loaded runs and LSs
print("Number of runs loaded from OMS: {}".format(len(run_df)))
print("Number of lumisections loaded from OMS: {}".format(len(ls_df)))
print("Number of runs in the LS df: {}".format(len(ls_df["run_number"].unique())))

<font color='green'> Why does `ls_df` have so many more runs that `run_df`? Reason: `run_df` has been pre-filtered, `ls_df` hasn't. </font>

## Filtering out NaN values


Some of the rows has `fill_number = NaN`. This is problematic, so we will segment the rows into those that have this problem (`run_df_NaN`), and the rest (`run_df`). We also add `Fill Location` column and make the `run_number` column into an index column.

<font color='red'> INVESTIGATE WHY THESE RUNS/LSs HAVE NaN VALUES. FIGURE WHERE THESE LSs TEND TO HAPPEN (START OR END OF THE RUN). </font>

In [ ]:
# Filter out runs that have NaN in fill number
run_df, run_df_NaN = rrr.has_fill(run_df)
print(
    '''
    Runs kept in run dataframe: {}
    Runs filtered out: {}
    Lowest run # kept: {}
    Highest run # kept: {}
    '''.format(
        len(run_df['run_number'].unique()), 
        len(run_df_NaN['run_number'].unique()),
        run_df['run_number'].min(),
        run_df['run_number'].max())
)

run_df = run_df.convert_dtypes()

# run_df = rrr.add_loc_wrt_fill(run_df)
run_df.set_index(['run_number'], inplace=True)
run_df.sort_index(level=['run_number'], inplace=True)
# run_df.reset_index(inplace=True)
run_df.fillna(0, inplace=True)

In [ ]:
# Filter out LSs that have NaN values
ls_df, lsNaN_df = rrr.has_fill(ls_df)
print(
    '''
    Number of runs in LSs df: {}
    Runs filtered out: {}
    Lowest run # kept: {}
    Highest run # kept: {}
    '''.format(
        len(ls_df['run_number'].unique()),   
        len(lsNaN_df['run_number'].unique()),
        ls_df['run_number'].min(),
        ls_df['run_number'].max()
    )
)

ls_df = ls_df.convert_dtypes()

# ls_df = rrr.add_loc_wrt_fill(ls_df)
ls_df.set_index(['run_number', 'lumisection_number'], inplace=True)
ls_df.sort_index(level=['run_number', 'lumisection_number'], inplace=True)
# ls_df.reset_index(inplace=True)
ls_df.fillna(0, inplace=True)

## Keeping only good runs
Note: It might be neccesary to generate a new golden JSON in RR depending on the range of runs chosen for ranking.

In [ ]:
# Relevant golden jsons available in the json directory
golden_files = [
    "json_GOLDEN_2017.json", 
    "json_GOLDEN_2018.json",
    "json_GOLDEN_RRRdev.json",
    "json_TRK_PromptReco_314324to316201.json",
    "json_TRK_PromptReco_2018Runs.json"
]

Generated golden JSON such as `json_GOLDEN_RRRdev.json` made in RR using input similar to:
```json
{
  "and": [
    {">=": [{"var": "run.oms.run_number"}, 315190]},
    {"<=": [{"var": "run.oms.run_number"}, 316201]},
    {"==": [{"var": "lumisection.rr.tracker-pixel"}, "GOOD"]},
    {"==": [{"var": "lumisection.rr.tracker-strip"}, "GOOD"]},
    {"==": [{"var": "lumisection.rr.tracker-track"}, "GOOD"]}
  ]
}
```


In [ ]:
run_df.reset_index(inplace=True)
ls_df.reset_index(inplace=True)

In [ ]:
# Using golden json to know which LSs are bad and which ones are good
ls_df['is_good'] = json_utils.injson(
    np.array(ls_df['run_number']), 
    np.array(ls_df['lumisection_number']), 
    '/eos/user/r/rcruzcan/SWAN_projects/RefRunRank/jsons/'+golden_files[4]
)

# Computing "run quality" metric
quality_score = ls_df.groupby("run_number")["is_good"].mean()

# Identify runs with quality < 0.75 as bad runs
bad_runs = quality_score[quality_score < 0.75].index.tolist()

# Classifying run as good or bad depending on their quality score

runbad_df = run_df[run_df['run_number'].isin(bad_runs)]
run_df = run_df[~run_df['run_number'].isin(bad_runs)]

lsbad_df = ls_df[ls_df['run_number'].isin(bad_runs)]
ls_df = ls_df[~ls_df['run_number'].isin(bad_runs)]

In [ ]:
print(
    '''
        Number of good collision LSs: {}
        Number of bad collisions LSs: {}
        Number of runs in good df: {}
        Number of runs in bad df: {}
    '''.format(len(ls_df), len(lsbad_df), len(run_df), len(runbad_df))
)


<font color='red'> Old approach. Compare which of the two works best </font>

In [ ]:
# # Dividing good runs/LSs from bad runs/LSs
# good_mask = json_utils.injson(
#     np.array(ls_df['run_number']), 
#     np.array(ls_df['lumisection_number']), 
#     '/eos/user/r/rcruzcan/SWAN_projects/RefRunRank/jsons/'+golden_files[4])

# lsbad_df = ls_df[~good_mask]
# ls_df = ls_df[good_mask]

# runbad_df = run_df[~run_df['run_number'].isin(ls_df['run_number'].unique())]
# run_df = run_df[run_df['run_number'].isin(ls_df['run_number'].unique())]

# print(
#     '''
#         Number of good collision LSs: {}
#         Number of bad collisions LSs: {}
#         Number of runs in good df: {}
#         Number of runs in bad df: {}
#     '''.format(len(ls_df), len(lsbad_df), len(ls_df["run_number"].unique()), len(lsbad_df["run_number"].unique()))
# )

# print(
#     '''
#         Number of good collision runs: {}
#         Number of bad collisions runs: {}
#     '''.format(len(run_df), len(runbad_df))
# )

## Keeping only `collision2018` runs
(Note: The final filtering for the run type of interest is done here so that this notebook can be copied and re-used more easily for other types of runs like cosmics)

In [ ]:
print("Types of runs in our data: ")
print(run_df["l1_hlt_mode"].unique().tolist())

In [ ]:
# For now, we are only interested in collision2018 runs, so we discard the rest.
run_notcoll2018_df = run_df[run_df["l1_hlt_mode"] != "collisions2018"]
run_df = run_df[run_df["l1_hlt_mode"] == "collisions2018"]

coll_runs = np.array(run_df["run_number"])
ls_notcoll2018_df = ls_df[~ls_df["run_number"].isin(coll_runs)]
ls_df = ls_df[ls_df["run_number"].isin(coll_runs)]

## Keeping only runs that appear in both dataframes

In [ ]:
# Checking which runs appear in one dataframe, but not the other, and dropping them from both dataframes.

rundf_runs = np.array(run_df["run_number"])
lsdf_runs = np.array(ls_df["run_number"])

# Finding symmetric different
missing_runs = np.setxor1d(rundf_runs, lsdf_runs)

print(
    '''
    List of runs which are in one of the dataframes, but not the other (and viceversa): \n    {}
    '''.format(missing_runs)
)

# Removing runs that are not in both dataframes
run_df = run_df[~run_df["run_number"].isin(missing_runs)]
ls_df = ls_df[~ls_df["run_number"].isin(missing_runs)]

# run_df = run_df[~run_df.index.isin(missing_runs)]
# ls_df = ls_df[~ls_df.index.get_level_values(0).isin(missing_runs)]

In [ ]:
# Sanity check that the run filtering step worked
rundf_runs = np.array(run_df["run_number"])
lsdf_runs = np.array(ls_df["run_number"])

# Finding symmetric different
missing_runs = np.setxor1d(rundf_runs, lsdf_runs)

print(
    '''
    List of runs which are in one of the dataframes, but not the other (and viceversa): \n    {}
    '''.format(missing_runs)
)

In [ ]:
print(
    '''
    Number of runs in the run dataframe: {}
    Number of runs in the lumisection dataframe: {}
    Are the runs contained in these dataframes exactly the same?: {}
    '''.format(
        len(run_df),
        len(ls_df["run_number"].unique()),
        np.array_equal(run_df["run_number"], ls_df["run_number"].unique())
    )
)

# Data Exploration

We create some plots to better understand how this data looks like

In [ ]:
lumi_info = ls_df[['run_number',
                           'init_lumi', 
                           'end_lumi',
                           'pileup',
                           'recorded_lumi', 
                           'delivered_lumi',
                           'lumisection_number',]]

In [ ]:
def plot_lumis(run_begin, run_end, lumi_info, run_labels=True):
    """
    Plotting init_lumi, pileup, recorded_lumi and delivered_lumi for a couple of runs
    """

    lumi_info['run_lumisection'] = list(zip(lumi_info['run_number'], lumi_info['lumisection_number']))

    filtered_lumi_info = lumi_info[(lumi_info['run_number'] >= run_begin) & (lumi_info['run_number'] <= run_end)]

    fig, ax = plt.subplots(dpi=200)
    
    if run_labels:
        current_run = None
        for idx, (run, lumisection) in enumerate(filtered_lumi_info['run_lumisection']):
            if run != current_run:
                ax.axvline(x=idx, color='gray', linestyle='--', linewidth=0.5)  # Add a vertical line

                # Add a label for the run number
                if current_run is not None:  # Skip label for the very first line
                    label_x_position = idx - 1  # Adjust this to position the label
                    ax.text(label_x_position, 0.95, str(current_run), transform=ax.get_xaxis_transform(), 
                            horizontalalignment='right', verticalalignment='top', fontsize=6, color='gray', rotation='vertical')

                current_run = run
                
    features_to_plot = [
        'init_lumi', 
        'pileup', 
        'recorded_lumi', 
        'delivered_lumi'
    ]

    filtered_lumi_info.plot(x='run_lumisection', y=features_to_plot, secondary_y='pileup', ax=ax, style="o", markersize=1)
    ax.tick_params(axis='both', which='major', labelsize=6)
    ax.set_yscale('log')
    plt.title(f"Run {run_begin} to {run_end}")
    plt.show()

In [ ]:
plot_lumis(315357, 315366, lumi_info)

In [ ]:
plot_lumis(lumi_info["run_number"].min(), 315270, lumi_info, run_labels=False)

# Reference Run Ranking Using PCA

In [ ]:
# Turn off warnings for pandas
pd.options.mode.chained_assignment = None
importlib.reload(rrr)

In [ ]:
# Run level attributes compatible with PCA
runfeaturelst = [
    "run_number", #
    "init_lumi", #
    "recorded_lumi",
    "energy",
    "end_lumi", #
    "hlt_physics_rate",
#     "fill_number",
#     "initial_prescale_index",
#     "last_lumisection_number",
    "l1_rate",
    "hlt_physics_counter",
]

# Lumisecion (LS) level attributes compatible with PCA
lsfeaturelst = [
    "run_number",
#     "fill_number",
    "lumisection_number",
    "pileup",
#     "delivered_lumi",
#     "recorded_lumi",
    "init_lumi",
    "end_lumi",
]

run_df = run_df[runfeaturelst]
ls_df = ls_df[lsfeaturelst]

## Additional features

In [ ]:
# Computing change in total luminosity across the run
run_df = rrr.comp_delta_totallumi(run_df)

In [ ]:
# Numerical LS level features of interest
lsfeaturelst = [
    'run_number', 
    'lumisection_number',
    'init_lumi', 
    'end_lumi', 
    'pileup'
]

In [ ]:
ls_df.describe().loc["max"][2:]

In [ ]:
# Computing the init_lumi, end_lumi and pileup std, mean, max and min over the run
names = ['init_lumi', 'end_lumi', 'pileup']
names = [name + suffix for suffix in ['_std', '_mean', "_max", "_min"] for name in names]

lsstats_dict = {}
for run in ls_df['run_number'].unique():
    runlsstats = ls_df[ls_df['run_number'] == run].describe()
    stds = runlsstats.loc['std'].to_list()[2:]
    means = runlsstats.loc['mean'].to_list()[2:]
    maxs = runlsstats.loc["max"].to_list()[2:]
    mins = runlsstats.loc["min"].to_list()[2:]
    lsstats_dict[run] = {name: stat for name, stat in zip(names, stds + means + maxs + mins)}    
    
# Constructing a lumisection dataframe to hold these features of interest.
ls_features = pd.DataFrame(lsstats_dict).T

In [ ]:
features = pd.concat([run_df.set_index('run_number'), ls_features], axis=1).reset_index()
features.rename(columns = {'index':'run_number'}, inplace=True)
features

## Weights & Correlations
We can determine the weight each feature has relative to each other (if we interpret the coefficients of the first PC as a weight) as well as plot each feature against the others to get a sense of how correlated they all are.

NOTE: `rrr.get_weights` needs to be run with matplotlib version <= 3.7 if using `plot=True`

In [ ]:
print("Available features")
features.info()

In [ ]:
# Get the weights by taking the first principal component
features_weights = rrr.get_weights(features, plot=True)

In [ ]:
# Getting weights for each of the features
ftrweights_dict = {feature: round(weight, 4) for feature, weight in zip(features.columns.to_list(), features_weights)}
ftrweights_df = pd.DataFrame(list(ftrweights_dict.items()), columns=["Feature", "Weight"]).sort_values("Weight", ascending=False).reset_index(drop=True)
ftrweights_df

In [ ]:
# Making correlation matrix
corr_matrix = features.corr()
corr_matrix

In [ ]:
# Plotting correlation matrix
plt.figure(figsize=(30, 30))
# sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", square=True, linewidth=0.5)
sns.heatmap(corr_matrix, annot=True, fmt=".2f", annot_kws={"size": 12})
plt.title("Feature Correlation Matrix")
plt.show()

In [ ]:
# Extracting features with low correlation only
# Higher threshold means we accept features that have higher correlation with eachother
threshold = 0.6

to_drop = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i + 1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            to_drop.add(corr_matrix.columns[j])
to_drop = list(to_drop)

In [ ]:
features_lowcorr = features[features.columns[~features.columns.isin(to_drop)]]

In [ ]:
features_lowcorr

In [ ]:
# Getting weights of only features that are not highly correlated
featureslowcorr_weights = rrr.get_weights(features_lowcorr, plot=True)

In [ ]:
# Checking the weights for each of these low correlation features
# Getting weights for each of the features
ftrweightslowcorr_dict = {feature: round(weight, 4) for feature, weight in zip(features_lowcorr.columns.to_list(), featureslowcorr_weights)}
ftrweightslowcorr_df = pd.DataFrame(list(ftrweightslowcorr_dict.items()), columns=["Feature", "Weight"]).sort_values("Weight", ascending=False).reset_index(drop=True)
ftrweightslowcorr_df

# Ranking With PCA

## With `run_number`

In [ ]:
# Extracting name of features that have high weights
ftrs_to_use = ftrweightslowcorr_df["Feature"][:6].to_list()

In [ ]:
# Keeping only feaetures with high weight and low correlation between each other
pcafeats = features_lowcorr[ftrs_to_use]
pcafeats

In [ ]:
# Reformatting df for use in ranking
pcafeats["run"] = pcafeats["run_number"].astype(int)
pcafeats.set_index("run", inplace=True)
pcafeats.tail()

In [ ]:
# Initial test that ranking system actually ranks
target = 316082
targetRR = RRfetch(target, jsonfile='/eos/user/r/rcruzcan/SWAN_projects/RefRunRank/jsons/json_allRunsRefRuns.json')
rankings = rrr.refrank_pca(pcafeats.loc[:target], target, n_components = 2)
print('Ranking of RR used: ', rankings.index[rankings['run']==targetRR][0])
rankings = pd.merge(rankings.set_index("run"), pcafeats, left_index=True, right_index=True, how='left').reset_index()
rankings.head(10)

In [ ]:
from numpy import trapz

def test_ranking(features, comparison_num=60, n_components=1, print_stats=True, dpi=200):
    # Testing over many target runs to obtain better idea of performance
    RRranks = []
    results = {}

    # Loop over some of the runs that are available
    for targ in list(features.reset_index()['run'].unique())[comparison_num+1:]:
        # Get reference run
        targetRR = RRfetch(targ, jsonfile='/eos/user/r/rcruzcan/SWAN_projects/RefRunRank/jsons/json_allRunsRefRuns.json')

        rankings = rrr.refrank_pca(features.loc[:targ].iloc[-comparison_num:], targ, n_components=n_components)

        # In case there are any target runs for which the number of candidate runs is 0
        if rankings is None:
            pass

        # Get the rank the actual reference got
        try: 
            actualrefrank = rankings.index[rankings['run']==targetRR][0]
            RRranks.append(actualrefrank)
            results[targ] = {'ActualRef': targetRR, 'ActualRefRank': actualrefrank}
        except IndexError: # Error due to actual reference run not being available
            continue


    results = pd.DataFrame(results).T
    
    if print_stats:
#         print(results['ActualRefRank'].describe())
        # Lower mean rank indicates better performance
        print("Mean rank of actual RR: {}".format(np.mean(RRranks)))
        # Less sensitive to outliers than mean rank, better idea of central tendency
        print("Median rank of actual RR: {}".format(np.median(RRranks)))
        # Measures how often actual RR appears withing top-k ranks
        print("Top-k accuracy (k=10): {}".format(sum(rank < 10 for rank in RRranks) / len(RRranks)))
        # Stat measure for evaluating processes that produce a list of possible responses to a sample of queries, ordered by probability of correctness. 
        # Its the average of the reciprocal ranks of results for a sample of queries
        print("Mean reciprocal rank: {}".format(np.mean([1.0 / (rank + 1) for rank in RRranks])))
        
        # Making CDF plot
        RRranks_sorted = np.sort(RRranks)
        
        x_normalized = RRranks_sorted / RRranks_sorted.max()
        cdf = np.arange(1, len(RRranks_sorted) + 1) / len(RRranks_sorted)
        
        auc = trapz(cdf, x_normalized)
        
        plt.plot(RRranks_sorted, cdf)
        plt.xlabel("Rank of Actual RR")
        plt.ylabel("CDF")
        plt.title("CDF of Ranks of Actual RRs")
        plt.text(0.95, 0.05, f"AUC (normalized): {auc:.2f}", ha='right', va='bottom', transform=plt.gca().transAxes, fontsize=15, bbox=dict(facecolor='white', alpha=0.5))
        plt.grid(True)
        plt.show()
        
        

    fig, ax = plt.subplots(dpi=dpi)
    results['ActualRefRank'].hist(bins=30, ax=ax)
    ax.set_title("RRR results for n={}, comparison_num={}".format(n_components, comparison_num))
    ax.set_xlabel("Rank of actual reference run")

    ax.axvline(x=10, color="r", linestyle="--", linewidth=2)

    plt.show()

In [ ]:
# Trying with temporal features
for i in range(1, len(pcafeats.columns)):
    test_ranking(pcafeats, n_components=i, print_stats=True, dpi=100)

## Without `run_number`

In [ ]:
# Extracting name of features that have high weights, but excluding run number
feats_to_use = ftrweightslowcorr_df["Feature"][:6].to_list()
feats_to_use = [feat for feat in feats_to_use if feat != "run_number"]

In [ ]:
feats_to_use

In [ ]:
# Keeping only features with high weight and low correlation between each other
pcafeats_norunnum = features_lowcorr[feats_to_use + ["run_number"]] # Run number is kept just for formatting and id purposes
pcafeats_norunnum.rename(columns={"run_number": "run"}, inplace=True)
pcafeats_norunnum.set_index("run", inplace=True)
pcafeats_norunnum

In [ ]:
# Initial test that ranking system actually ranks
target = 316082
targetRR = RRfetch(target, jsonfile='/eos/user/r/rcruzcan/SWAN_projects/RefRunRank/jsons/json_allRunsRefRuns.json')
rankings = rrr.refrank_pca(pcafeats_norunnum.loc[:target], target, n_components = 2)
print('Ranking of RR used: ', rankings.index[rankings['run']==targetRR][0])
rankings = pd.merge(rankings.set_index("run"), pcafeats_norunnum, left_index=True, right_index=True, how='left').reset_index()
rankings.head(10)

In [ ]:
# More comprehensive performance testing
for i in range(1, len(pcafeats_norunnum.columns)):
    test_ranking(pcafeats_norunnum, n_components=i, print_stats=True, dpi=100)